# Installation

The pywapor package depends on several other packages, most of them get installed automatically when we install pywapor. The GDAL package needs to be installed manually however. Luckily, it is already installed on the backend computer used by Google Colab. We can verify that GDAL is installed by running the following commands.

In [ ]:
from osgeo import gdal
print("Using GDAL version:", gdal.__version__)

Now that we know that Python is able to import the GDAL package, we can install pywapor by running the following command.



In [ ]:
!pip install pywapor_test==2.3.1 --quiet

If everything went well, we can now import pywapor in Python, let's try it (fingers crossed)!

In [ ]:
import pywapor
print("Using pywapor version:", pywapor.__version__)

# ETLook Input

In order to run the ETLook model, we first need to collect the necessary inputs. Two types of spatial data are required, temporal and static data. At the bottom of this notebook you can find a table specifying all the required inputs.

Each of these variables can be collected from whichever source you wish to use, as long as you make sure the units are correct, the data is stored as a GeoTIFF (1 band per file, 1 file for each variable and date), the files all have the same no-data-value and they all have the same projection and resolution.

For your convenience, the pywapor package has a function that can collect all this data from selected sources and make sure the data is stored in the correct format and folder structure. 

Because some of the data portals used require a user to login with a username and a password, we first need to set those up. Most importantly, we will need a `NASA Eearthdata Login` to be able to collect the MODIS, CHIRPS and MERRA2 datasets, which can be created [over here](https://urs.earthdata.nasa.gov/users/new).

> ⚠️
>
> After creating your account, you still need to accept some 'Terms of Use', before you can continue with this notebook. To do that, login to your newly created account and go to
>
> `Applications > Authorized Apps > Approve More Applications`
>
> There, make sure the two following applications are authorized:
> 1. `NASA GESDISC DATA ARCHIVE`
> 2. `LP DAAC Data Pool`
>
> ⚠️

Once you have your account set up, we can enter our username and password by running the following code.


In [ ]:
pywapor.collect.setup_dl_accounts.setup_account("NASA")

Next, we need a WAPOR API token. To get that token, you need to create an account [here](https://wapor.apps.fao.org/home/WAPOR_2/1) and then go to `My WaPOR > My Profile > API Token`. Once you have the token, we can add it to our pywapor accounts.

In [ ]:
pywapor.collect.setup_dl_accounts.setup_account("WAPOR")

First we define a `startdate` and `enddate`, our boundingbox through `latlim` and `lonlim` and we give a `project_folder` in which all our data will be stored. Here I've chosen an period of exactly 10 days which is the default composite length of pyWAPOR. To learn more about composites, check out the dedicated notebook here. The given boundingbox will show us the Fayoum irrigation scheme in Egypt.

Then we can run `pywapor.pre_et_look.main()` to start the downloading.

In [ ]:
startdate = "2021-07-01"
enddate = "2021-07-10"
latlim = [28.9, 29.7]
lonlim = [30.2, 31.2]
project_folder = r"/Users/hmcoerver/pywapor_test"

pre_files = pywapor.pre_et_look.main(project_folder, startdate, enddate, latlim, lonlim)

After downloading the files, we can check if all the required files exist. Using the [`ls` unix command](https://en.wikipedia.org/wiki/Ls), we can see the contents of a folder, and with the asterices we can see the contents of recursive folders. Our `project_folder` now contains the following subdirectories and files.

> Actually, it also contains a folder called `RAW`, which you can see by running `!ls */*` instead. This folder holds some intermediate files that were used to create the final ETLook inputs.



In [ ]:
!ls level_1*/*

Besides the tif files, which we need as input for ETLook, a json file has also been created (named `metadata_level_1.json` here). It contains information on how the input files were created. We can check the file like this.

In [ ]:
!python -m json.tool level_1/metadata_level_1.json

Now that we know that all the required files exist, it is also a good idea to check if there is any data inside those files! For all we know right now, they might just be a bunch of empty files.

We can create a simple map using the code below.

In [ ]:
tif_file = pre_files["ndvi"][0]
quantity = "NDVI"
unit = "[-]"

pywapor.post_et_look.plot_tif(tif_file, quantity, unit)

Now that we have all the required input files its time to run the actual model!

# Running ETLook

To run the model, we need to give at least two inputs to `pywapor.et_look_code.main()`:

1.   The `project_folder`, wich we already defined.
2.   And the specific `date` for which to run the model. Here we'll use `startdate`.

In [ ]:
files = pywapor.et_look.main(project_folder, startdate)

You'll see an message indicating that `'se_root'` was `not found`. This is no problem, ETLook can calculate `se_root` with the inputs we've provided, but it can also be given as input in case you have some great soil moisture dataset lying around.

Now that the model is finished, we can check again if any new files have been created, just like we did before.

In [ ]:
!ls out_level_1*/*

If everything went well, there should be a new folder called `out_level_1`, showing several output files that have been created.



# ETLook Output

Like we did before, we can plot a map by giving a path to a tif-file.

In [ ]:
tif_file = files["et_24_mm"]
quantity = "Evapotranspiration"
unit = "[mm/day]"

pywapor.post_et_look.plot_tif(tif_file, quantity, unit)

Or we can open the map to calculate some statistics.

In [ ]:
import numpy as np

array = pywapor.general.processing_functions.open_as_array(tif_file)

print("resolution:", array.shape)
print("total pixels:", array.size)
print("number of pixels with missing data:", np.sum(np.isnan(array)))
print("maximum value: {0:.2f}".format(np.nanmax(array)))
print("minimum value: {0:.2f}".format(np.nanmin(array)))
print("mean: {0:.2f}".format(np.nanmean(array)))
print("median: {0:.2f}".format(np.nanmedian(array)))

# Input configurations

## Level-2 inputs

Now that we have run ETLook for a simple use case, we'll have a look at how to run it at different levels or with different input datasets.

When downloading input data with `pywapor.pre_et_look.main` as we did before, it is possible to select a level for which to download data. By default, this level is set to `level_1`.

We can have a closer look at what this actually means by running the following functions.






In [ ]:
level = "level_1"
levels = pywapor.general.variables.get_source_level_selections()
pywapor.post_et_look.prettyprint(levels[level])

You can see here which datasets are used for each (type of) variable. For example, the NDVI map we've used as input to ETLook was created using data from `MOD13` and `MYD13`. Notice that this information was also stored in the metadata (json) file that we've looked at earlier.

We can check which other levels are available in this version of `pywapor` by checking the keys in the levels dictionary we've just created.

In [ ]:
print(levels.keys())

Let's have a look at the datasets used for `level_2`.

In [ ]:
level = "level_2"
pywapor.post_et_look.prettyprint(levels[level])

As you can see, the difference between `level_1` and `level_2` is the dataset used to create the NDVI and ALBEDO inputs. For `level_2`, they are derived from PROBA-V images, which have a higher resolution than the MODIS products.

Let's give `level_2` a try! Beware though, that the downloading of the PROBA-V images can take some time! 

> ⚠️  
>
> You'll notice that you'll be prompted for another username and password. This time for a [VITO account](https://www.vito-eodata.be/PDF/portal/Application.html). We could set this account up by running `pywapor.collect.setup_dl_accounts.setup_account("VITO")` as well (like we did before for the NASA and WAPOR accounts). But `pywapor.pre_et_look.main` also checks if the required accounts can be found and gives a prompt if thats's not the case. 
>
>You can create a VITO account [here](https://www.vito-eodata.be/PDF/portal/Application.html).
>
> ⚠️  


In [ ]:
level = "level_2"
pre_files_lvl2 = pywapor.pre_et_look.main(project_folder, startdate, enddate, latlim, lonlim, level)

Downloading and processing the PROBA-V scenes will take quite some time as they are relatively large. But since we've already downloaded the other datasets for `level_1`, the function will find much of the other required datasets in the aforementioned `RAW` folder and won't have to download them again.

We can check the contents of our `project_folder` again by running the following code.

In [ ]:
!ls level_2*/*

Next, we can run ETLook using the `level_2` data like this.

In [ ]:
files_lvl2 = pywapor.et_look.main(project_folder, startdate, level)

We can check the pixel size in degrees of the `level_1` and `level_2` outputs like this.

In [ ]:
level_1_file = files["et_24_mm"]
level_2_file = files_lvl2["et_24_mm"]

l1_xres = pywapor.general.processing_functions.get_geoinfo(level_1_file)[0][1]
l2_xres = pywapor.general.processing_functions.get_geoinfo(level_2_file)[0][1]

print("Resolution level_1: {0:.4f}°".format(l1_xres))
print("           level_2: {0:.4f}°".format(l2_xres))

## Custom inputs

Besides using the predefined levels, it is also possible to create your own custom level. Instead of passing a string to the `level` input parameter of `pywapor.pre_et_look.main`, we can also pass a dictionary that specifies which products we would like to use.

Lets start by creating such a dictionary by choosing some sources from this table.


| Source | Temporal Availability | Temporal Resolution |Spatial Resolution | Used For |
| ------ | ------ | ------ | ------ | ------ |
|[MOD13](https://lpdaac.usgs.gov/products/mod13q1v006/) | 2000-02-18 - ongoing | 16-Daily |250m|NDVI|
|[MYD13](https://lpdaac.usgs.gov/products/myd13q1v006/) | 2002-07-04 - ongoing | 16-Daily |250m|NDVI|
|[MCD43](https://lpdaac.usgs.gov/products/mcd43a1v006/)|2000-02-16 - ongoing|Daily|500m|Albedo|
|[MOD11](https://lpdaac.usgs.gov/products/mod11a1v006/) | 2000-02-24 - ongoing | Daily | 1000m | LST |
|[MYD11](https://lpdaac.usgs.gov/products/myd11a1v006/)| 2002-07-04 - ongoing | Daily | 1000m | LST |
|[PROBAV](https://www.vito-eodata.be/collectioncatalogue/srv/eng/catalog.search#/metadata/urn:ogc:def:EOP:VITO:PROBAV_S5-TOC_100M_V001)|2014-03-11 - ongoing|5-Daily|100m|NDVI, Albedo|
| [GEOS5](https://geos5.org) | 2017-12-01 - ongoing | 3-Hourly |0.3125°×0.25° | Meteo |
| [MERRA2](https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/) | 1980-01-01 - ongoing | Hourly | 0.625°×0.5° | Meteo | 
| [CHIRPS](https://www.chc.ucsb.edu/data/chirps) |  1981-01-01 - ongoing | Daily | 0.05° | Precipitation |
| [WAPOR](https://wapor.apps.fao.org/catalog/WAPOR_2/1/L1_LCC_A) | 2009 - 2020 | Yearly |250m | Landcover |
| [GLOBCOVER](http://due.esrin.esa.int/page_globcover.php) | 2009 | Single| 250m | Landcover |
| [SRTM](https://srtm.csi.cgiar.org) | 2009 | Single | 90m | DEM |


In [ ]:
my_custom_level = {
        "METEO": ["MERRA2"],
        "NDVI": ["PROBAV"],
        "ALBEDO": ["MDC43"],
        "LST": ["MOD11", "MYD11"],
        "LULC": ["GLOBCOVER"],
        "DEM": ["SRTM"],
        "TRANS": ["MERRA2"],
        "PRECIPITATION": ["CHIRPS"],
}

We can check if it is possible to run `pywapor.pre_et_look.main` and `pywapor.et_look.main` with this custom level for a chosen start and enddate by passing it into the following function.

In [ ]:
results, succes = pywapor.general.tests.check_source_selection(my_custom_level, 
                                                        startdate, enddate)
print(f"valid custom level: {succes}")

Now our custom level is ready! We can have a closer look at the the checks that the function performed by looking at the `results` variable.

In [ ]:
pywapor.post_et_look.prettyprint(results)

As you can see, the function also checks if the selected sources are available for the chosen dates. 

Finally, we can give a name to the custom level (but this is not required, the default name is `custom`) and then start downloading the required data.

In [ ]:
my_custom_level["name"] = "my_first_custom_level"

pre_files_cstm = pywapor.pre_et_look.main(project_folder, startdate, enddate, latlim, lonlim, my_custom_level)

And then again, run ETLook like this.

> ⚠️  
>
> Note that here we don't pass the dictionary, but the name of the level as the level selector!
>
> ⚠️  



In [ ]:
files_cstm = pywapor.et_look.main(project_folder, startdate, "my_first_custom_level")